In [9]:
from tqdm import tqdm
import cv2
import numpy as np
from demo.predictor import COCODemo
from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip

from PIL import Image

import argparse
import os

import torch

from torchvision import transforms as T
from torchvision.transforms import functional as F
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.data.datasets import FolderDataset
from maskrcnn_benchmark.engine.inference import inference
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.utils.collect_env import collect_env_info
from maskrcnn_benchmark.utils.comm import synchronize, get_rank
from maskrcnn_benchmark.utils.logger import setup_logger
from maskrcnn_benchmark.utils.miscellaneous import mkdir
from maskrcnn_benchmark.utils.imports import import_file

# from predictor import Cococompute_prediction, select_top_predictions, Resize
# Check if we can enable mixed-precision via apex.amp
try:
    from apex import amp
except ImportError:
    raise ImportError('Use APEX for mixed precision via apex.amp')


model = "R_50_C4_1x"
run = 19
Results = {}

config_file = f"configs/e2e_faster_rcnn_{model}.yaml" # args.config_file #"../configs/caffe2/e2e_mask_rcnn_R_50_FPN_1x_caffe2.yaml"

model = "R_50_C4_1x_pre"
# update the config options with the config file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cpu"])
#     cfg.merge_from_list(args.opts)

# cfg.DATASETS.DATA_DIR = data
cfg.TEST.IMS_PER_BATCH = 1 
cfg.TEST.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_HEADS.DETECTIONS_PER_IMG = 200 
cfg.MODEL.ROI_BOX_HEAD.NUM_CLASSES = 3 
cfg.MODEL.RETINANET.NUM_CLASSES = 3 

cfg.MODEL.ROI_HEADS.NMS = 0.1
# print(cfg)
    
    
    
    
    
data = f"/media/giancos/Football/CloudLabeling/Seeds_Striga_Strategy1/"
weight = f"/media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy1/"

coco_demo = COCODemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.5,
    weight_loading=f"{weight}/output/{model}_{run}/model_bestval.pth"
)

dataset = FolderDataset(data, "Testing")
COCODemo.CATEGORIES = dataset.CLASSES

image, target, index = dataset[9]

image = np.array(image)

predictions = coco_demo.compute_prediction(image)

Results.update({f"Pred_G": predictions[torch.nonzero(predictions.get_field("labels") == 1).squeeze(1)]})
Results.update({f"Pred_NG": predictions[torch.nonzero(predictions.get_field("labels") == 2).squeeze(1)]})
Results.update({f"GT_G": target[torch.nonzero(target.get_field("labels") == 1).squeeze(1)]})
Results.update({f"GT_NG": target[torch.nonzero(target.get_field("labels") == 2).squeeze(1)]})
    

    
    
    
    
    

data = f"/media/giancos/Football/CloudLabeling/Seeds_Striga_Strategy2/"
weight = f"/media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy2/"
run = 20

coco_demo = COCODemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.5,
    weight_loading=f"{weight}/output/{model}_{run}/model_bestval.pth"
)

dataset = FolderDataset(data, "Testing")
COCODemo.CATEGORIES = dataset.CLASSES

image, target, index = dataset[9]

image = np.array(image)

predictions = coco_demo.compute_prediction(image)

Results.update({f"Pred_R": predictions[torch.nonzero(predictions.get_field("labels") == 1).squeeze(1)]})
Results.update({f"Pred_S": predictions[torch.nonzero(predictions.get_field("labels") == 2).squeeze(1)]})
Results.update({f"GT_R": target[torch.nonzero(target.get_field("labels") == 1).squeeze(1)]})
Results.update({f"GT_S": target[torch.nonzero(target.get_field("labels") == 2).squeeze(1)]})
    


Loading weight from /media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy1//output/R_50_C4_1x_pre_19/model_bestval.pth.
['__background__', 'Germinated', 'Non-germinated']
Loading weight from /media/giancos/Football/CloudLabeling_DS/CloudLabeling/Seeds_Striga_Strategy2//output/R_50_C4_1x_pre_20/model_bestval.pth.
['__background__', 'Radicle', 'Seed']


In [10]:
import torch

weight_loading=f"{weight}/output/{model}_{run}/model_bestval.pth"
weigths = torch.load(weight_loading,map_location="cpu")
print(weigths.keys())
print(weigths['model']['roi_heads.box.predictor.cls_score.weight'].shape)

dict_keys(['model', 'optimizer', 'scheduler', 'iteration'])
torch.Size([3, 2048])


In [11]:
print(Results)

{'Pred_G': BoxList(num_boxes=41, image_width=2592, image_height=1944, mode=xyxy), 'Pred_NG': BoxList(num_boxes=32, image_width=2592, image_height=1944, mode=xyxy), 'GT_G': BoxList(num_boxes=44, image_width=2592, image_height=1944, mode=xyxy), 'GT_NG': BoxList(num_boxes=36, image_width=2592, image_height=1944, mode=xyxy), 'Pred_R': BoxList(num_boxes=50, image_width=2592, image_height=1944, mode=xyxy), 'Pred_S': BoxList(num_boxes=54, image_width=2592, image_height=1944, mode=xyxy), 'GT_R': BoxList(num_boxes=44, image_width=2592, image_height=1944, mode=xyxy), 'GT_S': BoxList(num_boxes=80, image_width=2592, image_height=1944, mode=xyxy)}


In [12]:

image_name = dataset.img_files[index].split("/")[-1]
print(image_name)



def createQualitative(image,keys, out, GT_only=False):
    result = image.copy()
 
    for key in keys:
    #     key = keys[1]
        # print(key)
        labels = Results[key].get_field("labels").tolist()
        boxes = Results[key].bbox

        color=[0.5]*3
        if "_G" in key: color=[17, 172, 222]
        if "_NG" in key: color=[32, 36, 143]
        if "_S" in key: color=[234, 140, 95]
        if "_R" in key: color=[134, 7, 3]
        if "GT_" in key and not GT_only: color=[0,0,0]


        for i_label, box in zip(labels, boxes):
            box = box.to(torch.int64)
            top_left, bottom_right = box[:2].tolist(), box[2:].tolist()
            result = cv2.rectangle(
                result, tuple(top_left), tuple(bottom_right), tuple(color), 5
            )



    result = Image.fromarray(result)
    folder=f"/home/giancos/Dropbox/Applicazioni/ShareLaTeX/SeedsResults/Qualitative/"
    # out = "_".join(keys) + ".jpg" #"Strat1.jpg"
    result.save(folder+out)
    
    
createQualitative(image, ["GT_G","Pred_G"], "G_GT_Pred.jpg")
createQualitative(image, ["GT_NG","Pred_NG"], "NG_GT_Pred.jpg")
createQualitative(image, ["GT_R","Pred_R"], "R_GT_Pred.jpg")
createQualitative(image, ["GT_S","Pred_S"], "S_GT_Pred.jpg")


createQualitative(image, ["Pred_R","Pred_S"], "Pred_R_S.jpg")
createQualitative(image, ["Pred_G","Pred_NG"], "Pred_NG_G.jpg")
createQualitative(image, ["GT_R","GT_S"], "GT_R_S.jpg",GT_only=True)
createQualitative(image, ["GT_G","GT_NG"], "GT_NG_G.jpg",GT_only=True)
    

image139.jpg
